##Find boroughs with word 'Toronto'

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0
!pip install folium

wiki='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
list=pd.read_html(wiki)
df=list[0]
df.columns=['PostalCode', 'Borough', 'Neighborhood']
df1=df.query('Borough != "Not assigned"').reset_index(drop=True)
df2=df1.groupby('PostalCode', as_index=False).agg(lambda x:','.join(set(x.dropna())))
df2.loc[df2['Neighborhood'] == "Not assigned", 'Neighborhood'] = df2['Borough']                                                  
df3=pd.read_csv("http://cocl.us/Geospatial_data")
df4=pd.concat([df2, df3], axis=1)
df4.drop("Postal Code", axis = 1, inplace=True)
df4.head()

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,West Hill,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [7]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, label in zip(df4['Latitude'], df4['Longitude'], df4['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
df5 = df4[df4['Borough'].str.contains("Toronto")]
print (df5)

   PostalCode           Borough  \
37        M4E      East Toronto   
41        M4K      East Toronto   
42        M4L      East Toronto   
43        M4M      East Toronto   
44        M4N   Central Toronto   
45        M4P   Central Toronto   
46        M4R   Central Toronto   
47        M4S   Central Toronto   
48        M4T   Central Toronto   
49        M4V   Central Toronto   
50        M4W  Downtown Toronto   
51        M4X  Downtown Toronto   
52        M4Y  Downtown Toronto   
53        M5A  Downtown Toronto   
54        M5B  Downtown Toronto   
55        M5C  Downtown Toronto   
56        M5E  Downtown Toronto   
57        M5G  Downtown Toronto   
58        M5H  Downtown Toronto   
59        M5J  Downtown Toronto   
60        M5K  Downtown Toronto   
61        M5L  Downtown Toronto   
63        M5N   Central Toronto   
64        M5P   Central Toronto   
65        M5R   Central Toronto   
66        M5S  Downtown Toronto   
67        M5T  Downtown Toronto   
68        M5V  Downt

In [12]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


In [14]:
map_dt = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df5['Latitude'], df5['Longitude'], df5['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt)  
    
map_dt

In [17]:
df5.loc[50, 'Neighborhood']
neighborhood_latitude = df5.loc[50, 'Latitude']
neighborhood_longitude = df5.loc[50, 'Longitude']
neighborhood_name = df5.loc[50, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


In [19]:
CLIENT_ID = 'I44Q3NWI4WWFMH5BMQP23Z0FGGFF0TD4J1HGPSZ4TGPBIBP4' 
CLIENT_SECRET = 'LCUELDEXV53QAL110ZNJ0RULFP3KMRLMUWROVQFVBBALFMAV' 
VERSION = '20180605' 
limit = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=I44Q3NWI4WWFMH5BMQP23Z0FGGFF0TD4J1HGPSZ4TGPBIBP4&client_secret=LCUELDEXV53QAL110ZNJ0RULFP3KMRLMUWROVQFVBBALFMAV&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

In [23]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d5bfaa05f9e790025ed42e2'},
 'response': {'headerLocation': 'Rosedale',
  'headerFullLocation': 'Rosedale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371},
   'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bae2150f964a520df873be3',
       'name': 'Mooredale House',
       'location': {'address': '146 Crescent Rd.',
        'crossStreet': 'btwn. Lamport Ave. and Mt. Pleasant Rd.',
        'lat': 43.678630645646535,
        'lng': -79.38009142511322,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.678630645646535,
          'lng': -79.380091425113

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Mooredale House,Building,43.678631,-79.380091
1,Rosedale Park,Playground,43.682328,-79.378934
2,Whitney Park,Park,43.682036,-79.373788
3,Alex Murray Parkette,Park,43.678300,-79.382773
4,Milkman's Lane,Trail,43.676352,-79.373842


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [26]:
downtowntoronto_venues = getNearbyVenues(names=df5['Neighborhood'],
                                   latitudes=df5['Latitude'],
                                   longitudes=df5['Longitude']
                                  )
print(downtowntoronto_venues.shape)
downtowntoronto_venues.head()

NameError: name 'getNearbyVenues' is not defined